In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
#Load the csv file
file_path="game.csv"
data=pd.read_csv(file_path)

In [3]:
data

,userId,game,rating
0,3,The Legend of Zelda: Ocarina of Time,4.0
1,6,Tony Hawk's Pro Skater 2,5.0
2,8,Grand Theft Auto IV,4.0
3,10,SoulCalibur,4.0
4,11,Grand Theft Auto IV,4.5
...,...,...,...
4995,4529,Donut County,2.5
4996,4533,MotorStorm: Apocalypse,3.0
4997,4544,The Last Guy,3.0
4998,4548,Valiant Hearts: The Great War,4.0


In [4]:
#Step 1: Create a user-item matrix (rows:users,colimns:games,values:rating
user_item_matrix=data.pivot_table(index="userId",columns="game",values="rating")
"""
pivot table : this function reshapes the dataset into matrix where:

       Each row represents a user (identified by userId).
       Each column represents a game (idenfied by game).
       The value in the matrix represent the ratings that 
       users gave to the games
    """

'\npivot table : this function reshapes the dataset into matrix where:\n\n       Each row represents a user (identified by userId).\n       Each column represents a game (idenfied by game).\n       The value in the matrix represent the ratings that \n       users gave to the games\n    '

In [6]:
#Step 2: Fill Nan Values with 0 (asuming no rating)
user_item_matrix_filled=user_item_matrix.fillna(0)
"""
This line replaces an missing valus by 0,
indicatesthat user did not rate that opaarticualar game
    """

'\nThis line replaces an missing valus by 0,\nindicatesthat user did not rate that opaarticualar game\n    '

In [16]:
#Step 3: Compute the cosine similarity between based on raw ratings
user_similarity=cosine_similarity(user_item_matrix_filled)

In [17]:
#Convert similarity matrix to a DataFrame for easy reference
user_similarity_df=pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

In [18]:
#Step 4: Functions to get game recommendations for a specific user based on
def get_collaborative_recommendation_for_user(user_id,num_recommendations=5):
    #Get the similarity scores for the input user with all other users
    similar_users=user_similarity_df[user_id].sort_values(ascending=False)
    #Get the similarity users (excluding the user themselves)
    similar_users=similar_users.drop(user_id)
    #Select the Top N similar users to limijt noise (e.g. Top 50 users
    top_similar_users=similar_users.head(50)
    #This select the top 50 most similar users to limit noise in the recommendation
    #Get rating of these similar users, Weighted by thier similarity cores
    weighted_ratings=np.dot(top_similar_users.values,user_item_matrix_filled.iloc[top_similar_users.index])
    #np.dot : This computes the dot prduct between the similarity scores of the top similar users and thier corresponding ratings in the user-item matrix.
    #The result is an array of weighted ratings for each game. 
    #Normalize by the sum of similarities
    sum_of_similarities=top_similar_users.sum()
    if sum_of_similarities >0:
        weighted_ratings/= sum_of_similarities
        #Tne weighted


    user_ratings=user_item_matrix_filled.iloc[user_id]
    unrated_games=user_ratings[user_ratings==0]
    #Identifies games that  the taregt user has not rated (i.e. rated 0)


    #Get the weighted score sfor unrated games
    game_recommendations=pd.Series(weighted_ratings, index=user_item_matrix_filled.columns).iloc[unrated_games.index]
    #This creates a pandas Series from the weighted ratings and filters it to include only the unrated games.
    #Finally, it sorts the recommendations in descending order and returns the top specified number of recommendations.

    #Return the top num_recommendations game recommendations
    return game_reccomendations.sort_values(ascending=False).head(num_recommendations)

#Example usage: Get recommendations for a user  with ID
recommended_games=get_collaborative_recommendation_for_user(3)
print("Recomended games for user 3:")
print(recommended_games)

IndexError: positional indexers are out-of-bounds

In [19]:
#step 4: fuction to get game recommendations for a specific user
def get_collaborative_recommendations_for_user(user_id,num_recommendations=5):
    #get similarity scores for the input user with all other users
    similar_users=user_similarity_df[user_id].sort_values(ascending=False)
    #get the most similar users (excluding the user themselves)
    similar_users=similar_users.drop(user_id)
    #select the top N simular users to limit noise (eg top 50 users)
    top_similar_users=similar_users.head(50)
    #get ratings of these similar users weighted by their similarity scores
    weighted_ratings= np.dot(top_similar_users.values,user_item_matrix_filled.loc[top_similar_users.index])
    #np.dot: this computed the dot product between the similarity scores of the top similar users
    #and their corresponding ratings in the user_item matrix
    #the result is an array of weighted ratings for each game

    #normalize by the sum of similarities
    sum_of_similarities=top_similar_users.sum()

    if sum_of_similarities>0:
        weighted_ratings/=sum_of_similarities
        #the weighted ratings are normalized by dividing by the sum of similarities to avoid biasing toward users with higher ratings.

    user_ratings=user_item_matrix_filled.loc[user_id]
    unrated_games=user_ratings[user_ratings==0]

    #get the weighted scores for the unrated games 
    game_recommendations=pd.Series(weighted_ratings, index=user_item_matrix_filled.columns).loc[unrated_games.index]
    #this creates a pandas series from the weighted ratings and filters to include only the unrated games
    #finally it sorts the recommended games in descending order and returns the top specified number of recommmendations

    #return the top 'num_recommendations' game recommendations
    return game_recommendations.sort_values(ascending=False).head(num_recommendations)



recommended_games=get_collaborative_recommendation_for_user(3)
print("Recomended games for user 3:")
print(recommended_games)

IndexError: positional indexers are out-of-bounds